In [2]:
!ls /kaggle/input/aksharantar/aksharantar_sampled/hin

ls: cannot access '/kaggle/input/aksharantar/aksharantar_sampled/hin': No such file or directory


In [1]:
!pip install lightning wandb 

Defaulting to user installation because normal site-packages is not writeable

[notice] A new release of pip is available: 23.0.1 -> 23.1
[notice] To update, run: pip install --upgrade pip


In [1]:
import torch
from torch import nn
import pandas as pd
import numpy as np
import torch.nn.functional as F
from torch.utils.data import DataLoader
import lightning.pytorch as pl
from torch import optim
from torch.utils.data.sampler import SubsetRandomSampler
from torch.utils.data import random_split
import wandb
from pytorch_lightning.loggers import WandbLogger
import random

In [2]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

In [3]:
# common_path = "/kaggle/input/aksharantar/aksharantar_sampled/hin/"
common_path = "./aksharantar_sampled/hin/"
train_path = common_path +'hin_train.csv'
test_path = common_path +'hin_test.csv'
val_path = common_path +'hin_valid.csv'

In [4]:
train_data = pd.read_csv(train_path)
val_data = pd.read_csv(val_path)

In [5]:
train_data.iloc[:, 1].values

array(['बिन्द्या', 'किरणकांत', 'यज्ञोपवीत', ..., 'असहमतों', 'सुलगायीं',
       'अंचुतेंगु'], dtype=object)

In [6]:
train_input = train_data.iloc[:, 0].values
val_input = val_data.iloc[:, 0].values
train_output = train_data.iloc[:, 1].values
val_output = val_data.iloc[:, 1].values

In [7]:
def get_dataloaders(path):
    data = pd.read_csv(path)
    data_input = data.iloc[:, 0].values
    data_output = data.iloc[:, 1].values
    return data_input, data_output

In [8]:
eng_to_int = {}
hin_to_int = {}
eng_to_int['SOW'] = 0
eng_to_int['EOW'] = 1
hin_to_int['SOW'] = 0
hin_to_int['EOW'] = 1
count = 2
for i in range(97, 123):
    eng_to_int[chr(i)] = count 
    count += 1
eng_to_int['PAD'] = count
count = 2
for i in range(2304, 2432):
    hin_to_int[chr(i)] = count
    count += 1
hin_to_int['PAD'] = count
eng_max_len = 26
hin_max_len = 22
def convert_word_to_tensor(word, language):
    if language == 'eng':
        char_to_int = eng_to_int
        max_len  =eng_max_len
    else:
        char_to_int = hin_to_int
        max_len = hin_max_len
    ascii = [[char_to_int['SOW']]]
    ascii += [[char_to_int[c]] for c in word]
    ascii.append([1]) # for end of word
    for x in range(max_len - len(ascii)):
        ascii.append([char_to_int['PAD']])
    ascii = torch.tensor(ascii)
    # one_hot = [torch.zeros(1, 1, 256) for _ in ascii]
    # for i, c in enumerate(ascii):
        # one_hot[i][0][0][c] = 1
    return ascii

In [9]:
chr(2309)

'अ'

In [10]:
convert_word_to_tensor('शस्त्रागार', 'hin')

tensor([[  0],
        [ 56],
        [ 58],
        [ 79],
        [ 38],
        [ 79],
        [ 50],
        [ 64],
        [ 25],
        [ 64],
        [ 50],
        [  1],
        [130],
        [130],
        [130],
        [130],
        [130],
        [130],
        [130],
        [130],
        [130],
        [130]])

In [11]:
convert_word_to_tensor('अ', 'hin')

tensor([[  0],
        [  7],
        [  1],
        [130],
        [130],
        [130],
        [130],
        [130],
        [130],
        [130],
        [130],
        [130],
        [130],
        [130],
        [130],
        [130],
        [130],
        [130],
        [130],
        [130],
        [130],
        [130]])

# Create dataloaders

In [12]:
class CustomDataset(torch.utils.data.Dataset):
    def __init__(self, input, output):
        self.input = input
        self.output = output
    def __len__(self):
        return len(self.input)
    def __getitem__(self, idx):
        return convert_word_to_tensor(self.input[idx], 'eng'), convert_word_to_tensor(self.output[idx], 'hin')

In [13]:
train_loader = CustomDataset(train_input, train_output)
val_loader = CustomDataset(val_input, val_output)
train_loader = DataLoader(train_loader, batch_size=128, shuffle=True)
val_loader = DataLoader(train_loader, batch_size=128, shuffle=True)

# Encoder

In [68]:
class Encoder(pl.LightningModule):
    def __init__(self, input_size, hidden_size, cell_type, dropout):
        super().__init__()
        self.hidden_size = hidden_size
        self.dropout = dropout
        self.embedding = nn.Embedding(input_size, hidden_size)
        self.rnn = self.find_cell_type(cell_type)(hidden_size, hidden_size, batch_first=True)
        self.dropout = nn.Dropout(dropout)
        

    def find_cell_type(self, cell_type):
        if cell_type == "GRU":
            return nn.GRU
        elif cell_type == "LSTM":
            return nn.LSTM
        else:
            return nn.RNN

    def forward(self, input, hidden):
        embedded = self.embedding(input).squeeze()
        output, hidden = self.rnn(embedded, hidden)
        return output, hidden

    def initHidden(self, batch_size=1):
        return torch.zeros(1, batch_size,  self.hidden_size, device=self.device)

# Decoder

In [69]:
class Decoder(pl.LightningModule):
    def __init__(self, output_size, hidden_size, cell_type, dropout):
        super().__init__()
        self.hidden_size = hidden_size
        self.dropout = dropout
        self.embedding = nn.Embedding(output_size, hidden_size)
        self.rnn = self.find_cell_type(cell_type)(hidden_size, hidden_size, batch_first=True)
        self.linear = nn.Linear(hidden_size, output_size)
        self.dropout = nn.Dropout(dropout)
        self.softmax = nn.Softmax(dim=1)

    def find_cell_type(self, cell_type):
        if cell_type == "GRU":
            return nn.GRU
        elif cell_type == "LSTM":
            return nn.LSTM
        else:
            return nn.RNN

    def forward(self, input, hidden):
        embedded = self.embedding(input).squeeze(2)
        output, hidden = self.rnn(embedded, hidden)
        # print("decoder: ", output, output.squeeze(0))
        output = self.softmax(self.linear(output.squeeze(0)))
        # output = torch.argmax(output, dim=1)
        return output, hidden

    def initHidden(self):
        return torch.zeros(1, 1, self.hidden_size, device=self.device)

# Train the encoder-decoder model

In [107]:
teacher_forcing_ratio = 1
def train(input_tensor, target_tensor, encoder:Encoder, decoder:Decoder, encoder_optimizer, decoder_optimizer, criterion, max_length=10):    
    encoder_optimizer.zero_grad()
    decoder_optimizer.zero_grad()

    input_length = input_tensor.size(0)
    target_length = target_tensor.size(0)
    encoder_hidden = encoder.initHidden(input_length)

    # encoder_outputs = torch.zeros(max_length, encoder.hidden_size, device=device)

    loss = 0

    encoder_output, encoder_hidden = encoder(
        input_tensor, encoder_hidden)
        # encoder_outputs[ei] = encoder_output[0, 0]

    decoder_input = torch.tensor([[[0]]] * target_length, device=device)

    decoder_hidden = encoder_hidden

    use_teacher_forcing = True if random.random() < teacher_forcing_ratio else False
    
    if use_teacher_forcing:
        # Teacher forcing: Feed the target as the next input
        for di in range(target_tensor.size(1)):
            decoder_output, decoder_hidden = decoder(
                decoder_input, decoder_hidden)
            # print(decoder_output.shape, target_tensor[:, di].shape)
            loss += criterion(decoder_output.squeeze(), target_tensor[:, di].squeeze())
            decoder_input = target_tensor[:, di].unsqueeze(1)  # Teacher forcing

    else:
        # Without teacher forcing: use its own predictions as the next input
        for di in range(target_length):
            decoder_output, decoder_hidden = decoder(
                decoder_input, decoder_hidden[di])
            topv, topi = decoder_output.topk(1)
            decoder_input = topi.squeeze().detach()  # detach from history as input
            loss += criterion(decoder_output, target_tensor[di].squeeze())
            if decoder_input.item() == 1:
                break
    print(loss)
    loss.backward()
    encoder_optimizer.step()
    decoder_optimizer.step()

    return loss.item() / target_length


In [108]:
import time
import math


def asMinutes(s):
    m = math.floor(s / 60)
    s -= m * 60
    return '%dm %ds' % (m, s)


def timeSince(since, percent):
    now = time.time()
    s = now - since
    es = s / (percent)
    rs = es - s
    return '%s (- %s)' % (asMinutes(s), asMinutes(rs))


In [109]:
def trainIters(encoder, decoder, n_iters, print_every=1000, plot_every=100, learning_rate=0.01, batch_size=128):
    start = time.time()
    plot_losses = []
    print_loss_total = 0  # Reset every print_every
    plot_loss_total = 0  # Reset every plot_every

    encoder_optimizer = optim.Adam(encoder.parameters(), lr=learning_rate)
    decoder_optimizer = optim.Adam(decoder.parameters(), lr=learning_rate)
    # training_pairs = [train_loader.__getitem__(random.randint(0, train_input.shape[0]))
                    #   for i in range(n_iters)]
    criterion = nn.CrossEntropyLoss()
    for iter in range(1, n_iters + 1):
        for input_tensor, target_tensor in train_loader:
            # training_pair = training_pairs[iter - 1]
            # input_tensor = training_pair[0]
            # target_tensor = training_pair[1]
            # print(input_tensor, target_tensor)
            loss = train(input_tensor, target_tensor, encoder,
                            decoder, encoder_optimizer, decoder_optimizer, criterion)
            print_loss_total += loss
            plot_loss_total += loss

        print_loss_avg = print_loss_total / print_every
        print_loss_total = 0
        print('%s (%d %d%%) %.4f' % (timeSince(start, iter / n_iters),
                                        iter, iter / n_iters * 100, print_loss_avg))



In [74]:
enc = Encoder(len(eng_to_int)+1, 64, "GRU", 0.1)
dec = Decoder(len(hin_to_int)+1, 64, "GRU", 0.1)

0m 21s (- 1m 27s) (1 20%) 0.3357
0m 48s (- 1m 12s) (2 40%) 0.3357
1m 12s (- 0m 48s) (3 60%) 0.3357
1m 36s (- 0m 24s) (4 80%) 0.3357
2m 1s (- 0m 0s) (5 100%) 0.3357


In [110]:
trainIters(enc, dec, 1)

tensor(107.4217, grad_fn=<AddBackward0>)
tensor(107.4217, grad_fn=<AddBackward0>)
tensor(107.4217, grad_fn=<AddBackward0>)
tensor(107.4217, grad_fn=<AddBackward0>)
tensor(107.4217, grad_fn=<AddBackward0>)
tensor(107.4217, grad_fn=<AddBackward0>)
tensor(107.4217, grad_fn=<AddBackward0>)
tensor(107.4217, grad_fn=<AddBackward0>)
tensor(107.4217, grad_fn=<AddBackward0>)
tensor(107.4217, grad_fn=<AddBackward0>)
tensor(107.4217, grad_fn=<AddBackward0>)
tensor(107.4217, grad_fn=<AddBackward0>)
tensor(107.4217, grad_fn=<AddBackward0>)
tensor(107.4217, grad_fn=<AddBackward0>)
tensor(107.4217, grad_fn=<AddBackward0>)
tensor(107.4217, grad_fn=<AddBackward0>)
tensor(107.4217, grad_fn=<AddBackward0>)
tensor(107.4217, grad_fn=<AddBackward0>)
tensor(107.4217, grad_fn=<AddBackward0>)
tensor(107.4217, grad_fn=<AddBackward0>)
tensor(107.4217, grad_fn=<AddBackward0>)
tensor(107.4217, grad_fn=<AddBackward0>)
tensor(107.4217, grad_fn=<AddBackward0>)
tensor(107.4217, grad_fn=<AddBackward0>)
tensor(107.4217,

KeyboardInterrupt: 

In [ ]:
def evaluate():
    input_length = input_tensor.size(0)
    target_length = target_tensor.size(0)
    encoder_hidden = encoder.initHidden(input_length)

    # encoder_outputs = torch.zeros(max_length, encoder.hidden_size, device=device)

    loss = 0

    encoder_output, encoder_hidden = encoder(
        input_tensor, encoder_hidden)
        # encoder_outputs[ei] = encoder_output[0, 0]

    decoder_input = torch.tensor([[[0]]] * target_length, device=device)

    decoder_hidden = encoder_hidden

    use_teacher_forcing = True if random.random() < teacher_forcing_ratio else False
    
    if use_teacher_forcing:
        # Teacher forcing: Feed the target as the next input
        for di in range(target_tensor.size(1)):
            decoder_output, decoder_hidden = decoder(
                decoder_input, decoder_hidden)
            # print(decoder_output.shape, target_tensor[:, di].shape)
            loss += criterion(decoder_output.squeeze(), target_tensor[:, di].squeeze())
            decoder_input = target_tensor[:, di].unsqueeze(1)  # Teacher forcing

    else:
        # Without teacher forcing: use its own predictions as the next input
        for di in range(target_length):
            decoder_output, decoder_hidden = decoder(
                decoder_input, decoder_hidden[di])
            topv, topi = decoder_output.topk(1)
            decoder_input = topi.squeeze().detach()  # detach from history as input
            loss += criterion(decoder_output, target_tensor[di].squeeze())
            if decoder_input.item() == 1:
                break


In [75]:
int_to_hin = {}
for x in hin_to_int.keys():
    int_to_hin[hin_to_int[x]] = x

In [106]:
word = "ghar"
tensor = convert_word_to_tensor(word, 'eng')
h0 = torch.zeros(1, 64, device=device)
print(tensor.shape, h0.shape)
enc_out, enc_hidden = enc(tensor, h0)
dec_in = torch.tensor([[0]], device=device)
dec_hidden = enc_hidden.view(1, 1, 64)
print(dec_in.shape, dec_hidden.shape)
for i in range(10):
    dec_out, dec_hidden = dec(dec_in, dec_hidden)
    topv, topi = dec_out.topk(1)
    index = torch.argmax(dec_out)
    dec_in = torch.tensor([[index.item()]], device=device)
    print(int_to_hin[index.item()], end="")

torch.Size([26, 1]) torch.Size([1, 64])
torch.Size([1, 1]) torch.Size([1, 1, 64])
५ॲॲ्ंिफ़ूूँ

In [88]:
int_to_hin[13]

'ऋ'